In [1]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [3]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [4]:
from problems.max_latency_problem import MaxLatencyProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
#requirements.min_throughput = 0.5e9
requirements.max_throughput = 50e9
requirements.max_latency = 5400

problem = MaxLatencyProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 1000, 'n_offsprings': 50, 'n_max_gen': 20000}

settings = {}

# NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization

ref_dirs = get_reference_directions("das-dennis", 4, n_partitions=14)
algo_settings['pop_size'] = int(np.ceil(len(ref_dirs) / 100) * 100)
algo_settings['n_offsprings'] = int(np.ceil(len(ref_dirs) / 100) * 100 / 4)

print("# dirs: %d" % len(ref_dirs))

settings['NSGA-III'] = {
    'label': 'nsga3',
    'algorithm': NSGA3(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=algo_settings['n_max_gen'],
                                                      n_max_evals=None)
}

setting = settings['NSGA-III']

# dirs: 680


In [5]:
termination = setting['termination']
algorithm = setting['algorithm']

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               #save_history=True,
               verbose=True
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     700 |  0.00000E+00 |  2.229620997 |      33 |            - |            -
    2 |     875 |  0.00000E+00 |  1.183360286 |      35 |  0.210708661 |        nadir
    3 |    1050 |  0.00000E+00 |  0.854018363 |      44 |  1.047542250 |        nadir
    4 |    1225 |  0.00000E+00 |  0.636133947 |      42 |  0.013570481 |        nadir
    5 |    1400 |  0.00000E+00 |  0.506083507 |      49 |  0.006929028 |        ideal
    6 |    1575 |  0.00000E+00 |  0.402428015 |      48 |  0.023956442 |            f
    7 |    1750 |  0.00000E+00 |  0.307913569 |      41 |  0.011268898 |        ideal
    8 |    1925 |  0.00000E+00 |  0.219895464 |      51 |  0.054290670 |            f
    9 |    2100 |  0.00000E+00 |  0.151445169 |      52 |  0.011596103 |        ideal
   10 |    2275 |  0.00000E+00 |  0.090082578 |      58 |  0.152491488 |        nadir
   11 |    2450 |  0.00000E+00 |  0.051819777 |      6

KeyboardInterrupt: 

In [ ]:
pickle.dump((problem, setting, res), open('max_latency_res_%s_%d_%d.pkl' % (setting['label'], algo_settings['pop_size'], algo_settings['n_max_gen']), 'wb'))


